<a href="https://colab.research.google.com/github/Yanlingwu95/Acoustic_Localization/blob/master/19_Extraction_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 授权绑定Google Drive 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 113597 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

# 指定当前的工作目录
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("drive/Acoustic_Localization/keras/data/data_19_1") 


# 查看文件目录，是否包含所需的文件
!ls

qd_LOS_32_berlin_20k.mat  qd_NLOS_32_berlin_20k.mat  v_LOS_32_20k.mat


In [0]:
import scipy.io as sio
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from keras import regularizers

Using TensorFlow backend.


In [0]:
#loading data
DDataFile = './data/data_Emre/v_LOS_32_20k.mat'
DData = sio.loadmat(DDataFile)
#get the coeff data
D_data = DData['h_coeff']
location_data = DData['positions']
#split the D_data and location_data and only need the first 2k
N = 2000
M = 32
d_data = D_data[:,:N]
loc_data = location_data[:,:N]
print("d_data shape", d_data.shape)
print("loc shape", loc_data.shape)


d_data shape (32, 2000)
loc shape (2, 2000)


In [0]:
I = np.eye(N)
D = np.fft.fft(I)
DH = np.conjugate(D)
print("I\n", I, I.shape)
print("D\n", D, D.shape)
print(D[3][2],D[2][3], D.shape)
print(DH[3][2],DH[2][3], DH.shape)

I
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]] (2000, 2000)
D
 [[1.        +0.j         1.        +0.j         1.        +0.j
  ... 1.        -0.j         1.        -0.j
  1.        -0.j        ]
 [1.        +0.j         0.99999507-0.00314159j 0.99998026-0.00628314j
  ... 0.99995559+0.00942464j 0.99998026+0.00628314j
  0.99999507+0.00314159j]
 [1.        +0.j         0.99998026-0.00628314j 0.99992104-0.01256604j
  ... 0.99982235+0.01884844j 0.99992104+0.01256604j
  0.99998026+0.00628314j]
 ...
 [1.        +0.j         0.99995559+0.00942464j 0.99982235+0.01884844j
  ... 0.99960031-0.02827057j 0.99982235-0.01884844j
  0.99995559-0.00942464j]
 [1.        +0.j         0.99998026+0.00628314j 0.99992104+0.01256604j
  ... 0.99982235-0.01884844j 0.99992104-0.01256604j
  0.99998026-0.00628314j]
 [1.        +0.j         0.99999507+0.00314159j 0.99998026+0.00628314j
  ... 0.99995559-0.0

In [0]:
DH = np.conjugate(D)
DH = np.transpose(DH)
# D = np.matrix(D) 
# DH = D.getH()
R = np.dot(DH, D)
R

array([[ 2.00000000e+03+0.00000000e+00j, -7.10542736e-14+7.10542736e-15j,
        -8.52651283e-14+1.06581410e-13j, ...,
        -1.13686838e-13-6.39488462e-14j, -1.42108547e-14-4.26325641e-14j,
        -7.10542736e-14+1.24344979e-14j],
       [-7.10542736e-14-7.10542736e-15j,  2.00000000e+03+0.00000000e+00j,
         2.70006240e-13+2.20268248e-13j, ...,
        -1.27897692e-13+8.52651283e-14j, -7.10542736e-14+1.56319402e-13j,
        -7.10542736e-14-1.77635684e-14j],
       [-8.52651283e-14-1.06581410e-13j,  2.70006240e-13-2.20268248e-13j,
         2.00000000e+03+0.00000000e+00j, ...,
        -2.84217094e-14+9.23705556e-14j,  0.00000000e+00+5.68434189e-14j,
        -7.10542736e-14+1.56319402e-13j],
       ...,
       [-1.13686838e-13+6.39488462e-14j, -1.27897692e-13-8.52651283e-14j,
        -2.84217094e-14-9.23705556e-14j, ...,
         2.00000000e+03+0.00000000e+00j,  2.84217094e-14-1.06581410e-14j,
         1.42108547e-13+8.52651283e-14j],
       [-8.52651283e-14+1.06581410e-13j, -9.